In [3]:
!pip install -U -q ultralytics
!pip install -U -q aifactory
!pip install -q tqdm

In [7]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True' 
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True  # 이미지 파일이 손상되었을 때 에러 발생 방지
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import numpy as np
from tqdm import tqdm
import shutil
import cv2
from ultralytics import YOLO
from ultralytics.utils.ops import nms_rotated


def get_imglist(dir="/workspace/dataset/"):
    imglist = [os.path.join(dir, f).replace("\\", "/") for f in os.listdir(dir) if f.endswith(('.png', '.npy'))]
    return imglist


def create_temp_dir(base_dir='./temp_cropped_patches'):
    
    # 임시 폴더가 이미 존재하면 삭제하고 새로 생성
    if os.path.exists(base_dir):
        shutil.rmtree(base_dir)
    os.makedirs(base_dir)
    return base_dir


def cluster_and_crop(image_path, k=3, size=1024, crop_num=43, threshold=0.7):
    # 이미지 로드 및 리사이즈
    img = Image.open(image_path)
    img_np = np.array(img)
    img_np = cv2.resize(img_np, (size, size), interpolation=cv2.INTER_LANCZOS4)
    
    
    # 이미지 데이터를 준비 (픽셀 수, 3) 형태로 변형
    img_data = img_np.reshape((-1, 3))
    img_data = np.float32(img_data)

    # K-Means 적용
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    _, labels, centers = cv2.kmeans(img_data, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    # 클러스터링 결과를 이미지로 변환
    centers = np.uint8(centers)
    segmented_img = centers[labels.flatten()]
    segmented_img = segmented_img.reshape(img_np.shape)

    # 검은색에 가장 가까운 클러스터 찾기
    black_cluster_index = np.argmin(np.linalg.norm(centers, axis=1))  # 가장 작은 RGB 값이 검은색
    
    # crop 영역 추출
    num_patches = []
    crop_size = segmented_img.shape[1] // crop_num
    for i in range(crop_num):
        for j in range(crop_num):
            
            crop_img = segmented_img[i * crop_size:(i + 1) * crop_size, j * crop_size:(j + 1) * crop_size]

            # 검은색 클러스터 비율 확인
            cluster_0_value = centers[black_cluster_index]
            cluster_0_mask = np.all(crop_img == cluster_0_value, axis=-1)
            if np.mean(cluster_0_mask) > threshold:
                num_patches.append((i, j))

    
    return num_patches

def save_cropped_patches_as_numpy(image_path, 
                                  crop_size, 
                                  resize_size,
                                  cluster_img_size,
                                  cluster_threshold,
                                  save_dir,
                                  is_cluster):
    image_name = os.path.basename(image_path)
    print(f"Processing {image_name}")
    image = Image.open(image_path).convert('RGB')
    
    image_width, image_height = image.size
    
    if is_cluster:
        print("Procssing Clustering...",end="")
        num_patches = cluster_and_crop(image_path, 
                                    k=3, 
                                    size=cluster_img_size, 
                                    crop_num=(image_width + crop_size - 1) // crop_size, 
                                    threshold=cluster_threshold)
        print("Done. ")
    else:
        num_patches = [(i, j) for i in range((image_width + crop_size - 1) // crop_size) for j in range((image_height + crop_size - 1) // crop_size)]
        
    for i, j in tqdm(num_patches):
        top_left_x = i * crop_size
        top_left_y = j * crop_size
        bottom_right_x = min(top_left_x + crop_size, image_width)
        bottom_right_y = min(top_left_y + crop_size, image_height)
        
        if bottom_right_x - top_left_x < crop_size:
            top_left_x = max(image_width - crop_size, 0)
            bottom_right_x = image_width
        if bottom_right_y - top_left_y < crop_size:
            top_left_y = max(image_height - crop_size, 0)
            bottom_right_y = image_height
        
        cropped_image = image.crop((top_left_x, top_left_y, bottom_right_x, bottom_right_y))
        cropped_image = cropped_image.resize((resize_size, resize_size), resample=Image.Resampling.LANCZOS)
        
        # 기존 코드에서 cropped_image를 numpy 배열로 변환
        # OpenCV를 사용하여 리사이즈
        # cropped_image = np.array(cropped_image)
        # cropped_image = cv2.resize(cropped_image, (resize_size, resize_size), interpolation=cv2.INTER_LANCZOS4)

        # NumPy 배열로 저장
        crop_filename = f"{os.path.splitext(image_name)[0]}_{top_left_x}_{top_left_y}.npy"  # NumPy로 저장
        crop_path = os.path.join(save_dir, crop_filename)
        np.save(crop_path, np.array(cropped_image, dtype=np.uint8))  # NumPy 배열로 저장 (dtype 명시)


    del image

def load_numpy_image(npy_path):
    return np.load(npy_path)

    
class CroppedPatchDataset(Dataset):
    def __init__(self, crop_image_paths, resize_size):
        self.crop_image_paths = crop_image_paths
        self.transform = transforms.ToTensor()
        self.resize_size = resize_size

    def __len__(self):
        return len(self.crop_image_paths)

    def __getitem__(self, idx):
        crop_image_path = self.crop_image_paths[idx]
        try:
            cropped_image = load_numpy_image(crop_image_path)
            cropped_image_tensor = self.transform(cropped_image)
        except Exception as e:
            print(f"Error loading image {crop_image_path}: {e}")
            return None  # 이미지 로드 실패 시 None 반환

        # 패치의 위치 정보를 반환
        image_name = os.path.basename(crop_image_path)
        name_parts = image_name.split('_')
        top_left_x = int(name_parts[-2])
        top_left_y = int(name_parts[-1].split('.')[0])
        position = torch.tensor([top_left_x, top_left_y])

        original_image_name = '_'.join(name_parts[:-2]) + '.png'

        return {
            'image_name': original_image_name,
            'image': cropped_image_tensor,
            'top_left_position': position
        }

def collate_fn(batch):
    batch = list(filter(lambda x: x is not None, batch))  # None 제거
    return torch.utils.data.dataloader.default_collate(batch)

def run_model(image_names, images, positions, model, scale_factor, device,result):
    # 모델 예측을 GPU에서 수행
    preds = model.predict(images, conf=0.15, save=False, device=device)  # 예측 결과: [batch_size]

    for img_name, pred, pos in zip(image_names, preds, positions):
        if img_name not in result:
            result[img_name] = []

        top_left_x, top_left_y = pos[0].item(), pos[1].item()

        # pred를 반복문 전에 CPU로 이동
        pred = pred.cpu()

        # 좌표 변환 및 conf 값 포함하여 저장
        for i in range(len(pred)):
            bbox = pred.obb.xywhr[i]  # [x, y, w, h, r]
            confidence = pred.obb.conf[i]  # confidence 값

            # bbox가 비어 있는 경우 해당 항목을 넘김
            if len(bbox) == 0:
                continue

            # 예측된 좌표를 원본 이미지 좌표로 변환
            x = bbox[0].item() * scale_factor + top_left_x
            y = bbox[1].item() * scale_factor + top_left_y
            w = bbox[2].item() * scale_factor
            h = bbox[3].item() * scale_factor
            r = bbox[4].item()  # 각도 값은 변환 불필요

            # 변환된 좌표와 confidence를 결과 리스트에 추가
            result[img_name].append({
                'xywhr': [x, y, w, h, r],
                'conf': confidence.item()
            })

In [ ]:
import os
import torch
from ultralytics import YOLO
from ultralytics.utils.ops import nms_rotated
from torch.utils.data import DataLoader

# 데이터 경로 설정
directory_path = '/workspace/dataset/'
# directory_path = './'
base_dir = './temp_cropped_patches'
img_list = get_imglist(directory_path)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 모델 설정
model_name = "./x_s512_best_args_65.pt"
crop_size = 196
resize_size = 512
cluster_img_size = 1024
cluster_threshold = 0.0
batch_size = 32*1024*1024 // resize_size**2  
divide_num = 54  # 원하는 분할 수로 설정

model = YOLO(model_name)

result = {}  # 최종 결과를 저장할 딕셔너리

length = len(img_list)
img_list_chunks = [img_list[i:i + length // divide_num] for i in range(0, length, length // divide_num)]

for chunk_idx, img_chunk in enumerate(img_list_chunks):
    # 임시 폴더 생성
    temp_dir = create_temp_dir(base_dir=base_dir)
    print(f"Processing chunk {chunk_idx + 1}/{len(img_list_chunks)}")

    # 현재 청크의 이미지별 결과 리스트 초기화
    for image_path in img_chunk:
        image_name = os.path.basename(image_path)
        result[image_name] = []

    # 모든 이미지를 크롭하여 임시 폴더에 저장
    for image_path in img_chunk:
        image_name = os.path.basename(image_path)
        print(f"Cropping image: {image_name}")
        # 패치 저장 시 일관된 이름 지정
        save_cropped_patches_as_numpy(image_path, 
                                          crop_size, 
                                          resize_size, 
                                          cluster_img_size,
                                          cluster_threshold,
                                          temp_dir,
                                          is_cluster=False)

    # 임시 폴더 내의 모든 패치 이미지 리스트 가져오기
    temp_list = get_imglist(temp_dir)

    # 데이터셋 및 DataLoader 설정
    dataset = CroppedPatchDataset(temp_list, resize_size=resize_size)
    dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=0)  # 필요에 따라 num_workers 조정

    for batch in dataloader:
        images = batch['image']
        positions = batch['top_left_position']
        patch_image_names = batch['image_name']

        # 모델 실행 및 결과 저장
        run_model(patch_image_names, images, positions, model, crop_size / resize_size, device, result)

    # 현재 청크의 모든 이미지에 대해 NMS 적용
    for image_path in img_chunk:
        image_name = os.path.basename(image_path)
        per_image_result = result[image_name]
        if per_image_result:
            # boxes와 scores 추출
            boxes = torch.tensor([pred['xywhr'] for pred in per_image_result])
            scores = torch.tensor([pred['conf'] for pred in per_image_result])

            # NMS 적용
            keep_indices = nms_rotated(boxes, scores, threshold=0.45)

            # NMS 결과를 최종적으로 업데이트
            result[image_name] = [per_image_result[i] for i in keep_indices]

    # 쿠다 캐시 제거 및 불필요한 메모리 제거
    torch.cuda.empty_cache()
    del dataset
    del dataloader
    # 임시 폴더 삭제
    shutil.rmtree(temp_dir)


Processing chunk 1/2
Cropping image: tasksmaple copy.png


100%|██████████| 1139/1139 [03:32<00:00,  5.35it/s]


Cropping image: tasksmaple copy 2.png


100%|██████████| 1139/1139 [03:50<00:00,  4.94it/s]



0: 768x768 (no detections), 32.5ms
1: 768x768 (no detections), 32.5ms
2: 768x768 32.5ms
3: 768x768 (no detections), 32.5ms
4: 768x768 (no detections), 32.5ms
5: 768x768 (no detections), 32.5ms
6: 768x768 32.5ms
7: 768x768 (no detections), 32.5ms
8: 768x768 (no detections), 32.5ms
9: 768x768 32.5ms
10: 768x768 (no detections), 32.5ms
11: 768x768 (no detections), 32.5ms
12: 768x768 (no detections), 32.5ms
13: 768x768 (no detections), 32.5ms
14: 768x768 (no detections), 32.5ms
15: 768x768 (no detections), 32.5ms
16: 768x768 (no detections), 32.5ms
17: 768x768 (no detections), 32.5ms
18: 768x768 (no detections), 32.5ms
19: 768x768 (no detections), 32.5ms
20: 768x768 (no detections), 32.5ms
21: 768x768 32.5ms
22: 768x768 32.5ms
23: 768x768 (no detections), 32.5ms
24: 768x768 (no detections), 32.5ms
25: 768x768 (no detections), 32.5ms
26: 768x768 (no detections), 32.5ms
27: 768x768 (no detections), 32.5ms
28: 768x768 (no detections), 32.5ms
29: 768x768 (no detections), 32.5ms
30: 768x768 (n

100%|██████████| 1139/1139 [03:32<00:00,  5.37it/s]


Cropping image: tasksmaple.png


 65%|██████▌   | 746/1139 [02:04<01:52,  3.51it/s]

In [2]:
import csv
import math

# 저장할 CSV 경로
csv_file = "./submission.csv"
data = []  # CSV에 저장할 데이터를 담을 리스트

# 이미지 이름별로 데이터 변환
for image_name, predictions in result.items():
    if not predictions:
        continue
    
    # 각 예측 결과를 변환하여 data 리스트에 추가
    for pred in predictions:
        cx, cy, width, height, angle = pred['xywhr']
        
        # 각도 변환: 라디안 -> 도(degrees)
        angle_deg = math.degrees(angle)
        if angle_deg < 0:
            angle_deg += 360
        
        # 예측 결과를 리스트로 추가
        data.append([image_name, cx, cy, width, height, angle_deg])

# CSV 파일로 저장
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # CSV의 헤더 작성
    writer.writerow(['image_name', 'cx', 'cy', 'width', 'height', 'angle'])
    
    # 각 행을 작성
    writer.writerows(data)

print(f"CSV 파일 '{csv_file}'이(가) 성공적으로 생성되었습니다.")


KeyboardInterrupt: 

In [5]:
print("Done")

Done


In [6]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
import aifactory.score as aif
import time

t = time.time()
aif.submit(model_name="top_196_512_00",
           key="128fd22e-34e1-4e7a-b9c9-3423c2e859ce")
print("time:", time.time() - t)

file : task.py
python
제출 완료
time: 21.081623792648315
